# Assignment 3: Decision Trees from scratch

__NOTE__ _This assignment is only required for students in 8515; students in 4510 may optionally complete it for extra credit._

For this assignment, your goal is to implement the ID3 algorithm (greedy information-gain based decision tree induction).  You should use the Congressional Voting Records data set, which I will show you how to load below.  This is a discrete multivariate dataset, which is well suited to decision trees.

Students in 4515 may optionally work in groups of 2 or 3 on this assignment (extra credit earned will be split equally); students in 8510 should work alone.

_Note that while scikit-learn does have decision trees, SKLearn in general doesn't really handle discrete data well, so the library version may not produce exactly the same trees that your algorithm will.  Also, since SKLearn expects numeric vector data, you'll need to do extra pre-processing before it will accept this data set._

Implement and test your algorithm; you should print the actual tree (a simple text-based visualization is fine), as well as calculating classification accuracy on held-out testing data.

As a reminder, please ensure that your notebook works correctly when you "restart kernel and run all."

In [1]:
# CSC 4510 - Machine Learning
# Assignment 3
# Scaffolding by Dr. Ben Mitchell
# Assignment completed by: <YOUR NAME(S) HERE>
# Resources used: 
#   <List any resources you used beyond the ones posted on Blackboard>
#   <This can include books, websites, other students, etc.>
#    https://en.wikipedia.org/wiki/ID3_algorithm
#    https://docs.scipy.org/doc/numpy/reference/
#    https://pandas.pydata.org/pandas-docs/stable/reference/index.html
#    https://homes.cs.washington.edu/~shapiro/EE596/notes/InfoGain.pdf
#    https://stackoverflow.com

## Read the data

The first thing we need to do is read the data from disk.  We'll use a library called Pandas to do that; it's really good for reading things like excel spreadsheets and comma-separated-value (CSV) data files.

In [2]:
# this time we'll use Pandas for reading our data file (it's great for CSVs)
import pandas as pd
# we'll also still want numpy
import numpy as np

### The Dataset
#### For this assignment, you'll be using a dataset covering congressional voting behavior.  The original version is downloadable from the UCI machine learning repository (https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records), but you can just grab a copy from Blackboard.  Here's the dataset description:

1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of
      Representatives Congressmen on the 16 key votes identified by the
      CQA.  The CQA lists nine different types of votes: voted for, paired
      for, and announced for (these three simplified to yea), voted
      against, paired against, and announced against (these three
      simplified to nay), voted present, voted present to avoid conflict
      of interest, and did not vote or otherwise make a position known
      (these three simplified to an unknown disposition).

5. Number of Instances: 435 (267 democrats, 168 republicans)

6. Number of Attributes: 16 + class name = 17 (all Boolean valued)

7. Attribute Information:
   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)

8. Missing Attribute Values: Denoted by "?"

   NOTE: It is important to recognize that "?" in this database does 
         not mean that the value of the attribute is unknown.  It 
         means simply, that the value is not "yea" or "nay" (see 
         "Relevant Information" section above).

In [3]:
# since the data file has no header, we need to define "names" for the columns
# since the class label (political party in this case) is first, we'll assign
# that name, and then just label the various votes as vN, where N is a number
colNames = ['party']
for i in range(16):
    colNames.append('v'+str(i))
    
# actually read the data, then take a look at it
raw = pd.read_csv('house-votes-84.data', header=None, names=colNames )
raw

,party,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
9,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [4]:
# as usual, length will give us the number of examples
len(raw)

435

## Split the data into training and testing sets

Next, we need to split the data into train and test sets, and then break off the labels from the feature vectors.  We'll also convert from Pandas style tables to numpy style arrays.

In [5]:
# let's shuffle this and split it into train and test sets:
shuffled = raw.sample(frac=1) # randomly re-order the examples
trainFrac = 0.7 # 70%/30% train/test split
trainCount = int(len(raw) * trainFrac)
train = shuffled[:trainCount]
test = shuffled[trainCount:]

In [6]:
print("train example count:", len(train), ", test example count:", len(test))

train example count: 304 , test example count: 131


In [7]:
test

,party,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15
149,democrat,n,n,y,n,n,n,y,y,y,y,n,n,y,n,y,y
388,democrat,n,y,y,y,y,y,n,n,n,n,n,y,y,y,n,?
99,republican,n,n,n,y,y,y,n,n,n,y,?,y,y,y,n,n
331,democrat,y,?,y,n,n,n,y,y,y,n,n,n,n,n,y,?
183,democrat,?,?,?,?,?,?,?,?,y,?,?,?,?,?,?,?
246,democrat,n,n,y,n,y,n,y,y,y,n,n,n,n,y,?,y
324,republican,n,y,n,y,y,y,n,n,n,n,y,y,y,y,n,n
325,democrat,n,y,n,n,y,y,n,n,?,n,n,y,y,y,n,y
130,democrat,y,?,y,n,?,?,y,y,y,n,n,n,n,n,y,?
296,republican,n,n,y,y,y,y,n,n,n,y,n,y,y,y,y,y


In [8]:
test['party'].values

array(['democrat', 'democrat', 'republican', 'democrat', 'democrat',
       'democrat', 'republican', 'democrat', 'democrat', 'republican',
       'democrat', 'republican', 'democrat', 'democrat', 'republican',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'republican', 'democrat', 'democrat', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'democrat', 'democrat', 'republican',
       'democrat', 'democrat', 'republican', 'democrat', 'democrat',
       'republican', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
       'republican', 'democrat', 'democrat', 'republican', 'democrat',
       'democrat', 'republican', 'republican', 'democrat', 'democrat',
       'democrat

In [9]:
# split off class labels and convert to numpy array
trainLabels = train['party'].values
testLabels = test['party'].values
len(trainLabels)

304

In [10]:
# let's take a look at what we got
trainLabels

array(['republican', 'republican', 'republican', 'democrat', 'republican',
       'republican', 'democrat', 'democrat', 'republican', 'republican',
       'republican', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'republican', 'republican', 'republican',
       'republican', 'republican', 'republican', 'democrat', 'democrat',
       'democrat', 'republican', 'democrat', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'republican', 'republican', 'democrat',
       'democrat', 'democrat', 'democrat', 'democrat', 'republican',
       'democrat', 'democrat', 'democrat', 'democrat', 'democrat',
       'democrat', 'republican', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'republican', 'republican', 'democrat', 'democrat',
       'democrat', 'democrat', 'democrat', 'republican', 'democrat',
    

In [11]:
# find all the distinct options:
classLabels = np.unique(trainLabels)
classLabels

array(['democrat', 'republican'], dtype=object)

In [12]:
# now remove the labels from the feature vectors and convert to numpy array
trainFeatures = train.drop(['party'], axis=1).values
testFeatures = test.drop(['party'], axis=1).values
trainFeatures

array([['n', 'n', 'n', ..., 'y', 'n', 'n'],
       ['n', 'n', 'n', ..., 'y', 'y', 'y'],
       ['n', 'y', 'n', ..., 'y', 'n', '?'],
       ...,
       ['y', 'y', 'y', ..., 'n', 'y', '?'],
       ['y', 'n', 'y', ..., '?', 'y', 'y'],
       ['n', 'n', 'n', ..., 'y', 'n', 'n']], dtype=object)

In [13]:
# we can look at a row by index
trainFeatures[0]

array(['n', 'n', 'n', 'y', 'y', 'y', 'n', 'n', 'n', 'n', 'n', 'y', 'y',
       'y', 'n', 'n'], dtype=object)

In [14]:
# we can look at a single feature with a double index (row, col)
trainFeatures[0][0]

'n'

In [15]:
# total number of training examples
numExamples = len(trainFeatures)
numExamples

304

In [16]:
# number of features in the first example (should be the same for all examples)
numFeatures = len(trainFeatures[0])
numFeatures

16

In [17]:
# now let's find the unique feature values
featureValues = np.unique(trainFeatures)
featureValues

array(['?', 'n', 'y'], dtype=object)

# Implement Decision Tree #

From here, you're on your own to implement an algorithm to train and test a decision tree on this dataset.

In [18]:
# Helpful variables
party = train.keys()[0]
eps = np.finfo(float).eps
unique_class_labels = train[party].unique()

In [19]:
# get average entropy
def avg_entropy(dataset, attr):
    unique_features = dataset[attr].unique()
    avg_attr_entropy = 0
    for feature in unique_features:
        child_entropy = 0
        for class_label in unique_class_labels:
            
            attr_count = len(dataset[attr][dataset[attr]==feature][dataset[party]==class_label])
            subsetset_count = len(dataset[attr][dataset[attr]==feature])
            
            conditional_prob = attr_count / subsetset_count
            child_entropy += -conditional_prob*np.log2(conditional_prob+eps)
            
        set_conditional_prob = subsetset_count/len(dataset)
        avg_attr_entropy += -set_conditional_prob*child_entropy
    return abs(avg_attr_entropy)

In [20]:
def find_entropy(dataset):
    entropy = 0
    attributes = dataset[party].unique()
    for attribute in attributes:
        fraction = dataset[party].value_counts()[attribute]/len(dataset[party])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [21]:
find_entropy(train)

0.977118106492721

In [22]:
def find_best_partition(dataset):
    entropy_attr = []
    info_gain = []
    for key in dataset.keys()[1:]:
        info_gain.append(find_entropy(dataset)-avg_entropy(dataset, key))
    return dataset.keys()[1:][np.argmax(info_gain)]

In [23]:
find_best_partition(train)

'v3'

In [24]:
def get_subtable(dataset, node, value):
    return dataset[dataset[node]==value].reset_index(drop=True)

In [25]:
print(get_subtable(get_subtable(train, 'v2', '?'), 'v3', '?'))

        party v0 v1 v2 v3 v4 v5 v6 v7 v8 v9 v10 v11 v12 v13 v14 v15
0  republican  n  ?  ?  ?  ?  ?  ?  ?  ?  ?   ?   ?   ?   y   ?   ?
1  republican  ?  ?  ?  ?  ?  ?  ?  ?  ?  ?   ?   ?   ?   ?   ?   ?
2    democrat  y  y  ?  ?  ?  y  n  n  n  n   y   n   y   n   n   y
3    democrat  ?  ?  ?  ?  n  y  y  y  y  y   ?   n   y   y   n   ?
4  republican  ?  ?  ?  ?  n  y  n  y  y  n   n   y   y   n   n   ?


In [26]:
def buildTree(dataset, tree=None):    
    
    # Create root node
    node = find_best_partition(dataset)
    attr_value = np.unique(dataset[node])
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    for value in attr_value:
        # get subset for this iteration
        subtable = get_subtable(dataset, node, value)
        class_values, counts = np.unique(subtable['party'], return_counts=True)
        
        # if node has only 1 type of attribute, return node with that attribute
        if len(counts)==1:
            tree[node][value] = class_values[0]
        
        # else, create new child node
        else:
            tree[node][value] = buildTree(subtable)
    return tree

In [27]:
myTree = buildTree(train)
myTree

{'v3': {'?': {'v11': {'?': 'republican', 'n': 'democrat', 'y': 'republican'}},
  'n': {'v2': {'?': 'democrat',
    'n': {'v5': {'?': 'democrat',
      'n': {'v1': {'n': 'republican', 'y': 'democrat'}},
      'y': 'democrat'}},
    'y': 'democrat'}},
  'y': {'v10': {'?': 'republican',
    'n': {'v14': {'?': 'republican',
      'n': 'republican',
      'y': {'v9': {'n': {'v15': {'?': {'v1': {'n': 'democrat',
            'y': 'republican'}},
          'n': 'republican',
          'y': 'democrat'}},
        'y': 'republican'}}}},
    'y': {'v8': {'n': {'v15': {'?': {'v0': {'n': 'republican',
          'y': 'democrat'}},
        'n': {'v0': {'n': {'v9': {'n': 'democrat',
            'y': {'v2': {'n': 'republican', 'y': 'democrat'}}}},
          'y': 'republican'}},
        'y': 'republican'}},
      'y': {'v2': {'?': 'democrat',
        'n': 'democrat',
        'y': {'v0': {'n': 'democrat', 'y': 'republican'}}}}}}}}}}

In [28]:
import pprint
pprint.pprint(myTree)

{'v3': {'?': {'v11': {'?': 'republican', 'n': 'democrat', 'y': 'republican'}},
        'n': {'v2': {'?': 'democrat',
                     'n': {'v5': {'?': 'democrat',
                                  'n': {'v1': {'n': 'republican',
                                               'y': 'democrat'}},
                                  'y': 'democrat'}},
                     'y': 'democrat'}},
        'y': {'v10': {'?': 'republican',
                      'n': {'v14': {'?': 'republican',
                                    'n': 'republican',
                                    'y': {'v9': {'n': {'v15': {'?': {'v1': {'n': 'democrat',
                                                                            'y': 'republican'}},
                                                               'n': 'republican',
                                                               'y': 'democrat'}},
                                                 'y': 'republican'}}}},
                      'y': {'v

In [42]:
def predict(row, tree):
    for node in tree.keys():
        value = row[node]
        tree = tree[node][value]
        if type(tree) is dict:
            prediction = predict(row, tree)
        else:
            prediction = tree
            break
    return prediction

In [43]:
def find_predictions(dataset, tree):
    positives = 0
    for i in range(len(dataset)):
        row = dataset.iloc[i]
        prediction = predict(row, tree)
        if (prediction == dataset.iloc[i][0]):
            positives += 1
    return positives / len(dataset)

In [44]:
# Prediction for training data
find_predictions(train, myTree)

1.0

In [45]:
# Prediction for testing data
find_predictions(test, myTree)

0.9312977099236641

In [46]:
# import tree classifier and preprocessing utility
from sklearn import tree
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
clf = tree.DecisionTreeClassifier()

In [47]:
# Encode categorical data into numbers 
def encode_labels(label):
    le.fit(label)
    encodedLabels = le.transform(label) 
    return encodedLabels

In [48]:
encode_labels(trainLabels)

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,

In [49]:
# A bit tricker for table. Encode for each row individually
def encode_features(features):
    encodedFeatures = []
    for x in features:
        le.fit(x)
        encodedFeatures.append(le.transform(x))
    return encodedFeatures

In [50]:
encode_features(trainFeatures)

[array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]),
 array([0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]),
 array([1, 2, 1, 2, 2, 2, 1, 0, 1, 1, 0, 0, 0, 2, 1, 0]),
 array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1]),
 array([2, 1, 2, 1, 1, 1, 0, 2, 2, 0, 1, 1, 1, 1, 2, 0]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]),
 array([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1]),
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]),
 array([1, 2, 1, 2, 2, 2, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0]),
 array([1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 0]),
 array([0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1]),
 array([1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1]),
 array([0, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 0, 1, 1, 2, 0]),
 array([0, 1, 

In [51]:
# Fit data using encoded labels
clf.fit(encode_features(trainFeatures), encode_labels(trainLabels))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [52]:
# Predict using training features
clf.score(encode_features(trainFeatures), encode_labels(trainLabels))

1.0

In [53]:
# Predict using test data
skTree = clf.score(encode_features(testFeatures), encode_labels(testLabels))
skTree

0.916030534351145